<img src="http://www.exalumnos.usm.cl/wp-content/uploads/2015/06/Isotipo-Negro.gif" title="Title text" width="20%" height="20%" />


<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 1 - Redes Neuronales y *Deep Learning* </H3>
<hr style="height:2px;border:none"/>

**Temas**  

* Entrenamiento de redes *Feed-Forward* vı́a GD y variantes (SGD, mini-*batches*), *momentum*, regularización y tasa de aprendizaje adaptiva.
* Rol de capas ocultas y mayor profundidad (*Deep Learning*).
* Diseño y entrenamiento de Redes Neuronales Convolucionales (CNNs).
* Aplicaciones de las Redes Neuronales Convolucionales
* Técnicas de regularización: *Dropout* y *Batch Normalization* 

<hr style="height:2px;border:none"/>
La tarea se divide en secciones:

1. Red Neuronal *Feed Forward* para Detectar Exoplanetas  
2. *Deep Networks*  
3. Redes Convolucionales en Imágenes  
4. CNN *vs* RNN Prediciendo el Ozono Atmosférico

<a id="cuarto"></a>
## 4. CNN *vs* RNN Prediciendo el Ozono Atmosférico
---

En esta actividad utilizaremos redes neuronales para predicción de series de tiempo. Dada una serie de registros del valor de una variable en el tiempo (serie de tiempo) nos interesa entrenar un modelo que pueda predecir los valores futuros de la misma serie. 

En este caso, la serie corresponde a las concentraciones diarias (máximas) de $O_3$ medidas en la comuna de Las Condes entre Diciembre de 2003 y Diciembre de 2016. El $O_3$ (a nivel respirable) es un contaminante con graves efectos sobre la salud humana y sobre la agricultura, por lo que monitorear y predecir sus concentraciones en el tiempo es de gran relevancia. Se sabe que el $0_3$ alcanza sus niveles máximos en verano, por lo que estudiaremos sólo esos períodos. Esto hace que la serie que usted recibirá no es totalmente continua en el tiempo en el sentido de que los registros se miden hasta marzo de un año y se reinician en noviembre de ese mismo año. **Por favor considere atentamente esta situación.**   

El archivo *CSV* que usted recibirá tiene las siguientes columnas 

[‘registered_on’, ‘CO', 'PM10', 'PM25', 'NO', 'NOX', 'WD', 'RH', 'TEMP', 'WS', 'UVA', 'UVB', 'O3']

El primero corresponde a la fecha del registro. La última columna corresponde a la variable que queremos predecir y las demás son mediciones de otros contaminantes y de variables meteorológicas que podrían ayudar a predecir los niveles de $O_3$.  

Su tarea consistirá en entrenar una red capaz de predecir los niveles de ozono el día de mañana sólo partir de los niveles *previas* de ozono y demás variables. Para ésto pre-procesaremos los datos, cada contaminante tendrá su serie de tiempo y asociada con las otras. Con ésto deberemos generar *secuencias de entrenamiento*, recordar que cada secuencia estará asocaida a un *Target* que es el $O_3$ del día siguiente.

Compararemos y aplicaremos redes convolucionales uni-dimensionales y redes neuronales recurrentes, las cuales se adaptan perfectamente a escenarios de secuencias ya que modelan que el estado actual se genera a partir del estado anterior (como en una cadena de Markov).


**a)** Explore los datos y visualícelos, ya sea a través de medidas de tendencia o gráficos. Lo primero que deberá realizar será crear su conjunto de validación/test a partir de los datos, por ejemplo las últimas mediciones puesto que la tarea será predecir los datos futuros (*out of box*). Además escale los datos apropiadamente.
#### Solo con input: Ozono (1-D)
Considere para construir las secuencias de entrenamiento sólo la información previa del $O_3$

**b)** Ahora nos gustarı́a manipular los datos, para que hagamos la predicción para el tiempo siguiente usando los valores de los últimos perı́odos de tiempo. El número de perı́odos de tiempos "hacia atrás" que usaremos se denomina *lag*. Por ejemplo, tendremos un *lag* igual a 3, si para predecir el valor $x_{t+1}$ en el tiempo siguiente usamos la información del tiempo actual $x_t$ y la de los dos perı́odos anteriores $x_{t-1}$ y $x_{t-2}$ como variables de entrada. Realice una función que reciba una secuencia de valores y la transforme en dos arreglos *dataX* (inputs) y *dataY* (targets), además utilicela para generar los conjuntos de entrenamiento y test para el problema decidiendo el valor del *lag*.

In [ ]:

def create_dataset(dataset,lag=1):
    return np.array(dataX),np.array(dataY)

# Por ejemplo si en el dataset tenemos el arreglo 20.7,17.9,18.8,14.6,15.8,15.8,10.1.

create_dataset(dataset,3)

'''
La función debiese generar $(X_1,X_2,X_3)$ e $Y$:

|$X_0$|$X_1$|$X_2$|Y|
|---|---|---|---|
|20.7|17.9|18.8|14.6|
|17.9|18.8|14.6|15.8|
|18.8|14.6|15.8|15.8|
|14.6|15.8|15.8|10.1|
'''

# lag = #your call
trainX, trainY = create_dataset(stream_train_scaled, lag)


**c)** En estos momentos tenemos nuestros datos en la forma [*samples, features*]. Sin embargo, la red RNN necesita que los datos se encuentren en un arreglo de tres dimensiones [*samples, time steps, features*]. Transforme el  conjuntos de entrenamiento y de pruebas a la estructura deseada, incorporando el *lag* como *features*. Defina dos modelos para comparar, una red recurrente simple y una red convolucional unidimensional (donde el kernel es un vector de largo $w$), comente sobre las dimensiones/*shape* de los parámetros de cada una de las redes. Entrene los dos modelos con la nueva representación ¿Qué limitaciones tiene el tamaño/*width* del kernel de la convolucional?.

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN
model_rnn = Sequential()
model_rnn.add(SimpleRNN(units=4, input_shape=(1,lag), activation='tanh',return_sequences=False,go_backwards=False))
model_rnn.add(Dense(1,activation='linear'))
model_rnn.compile(loss='mean_squared_error', optimizer='adam')
model_rnn.summary()
rnn_weights = model_rnn.get_weights()
model_rnn.fit(trainX, trainY, epochs=25, batch_size=1, verbose=1)

#and now CNN
model_cnn = Sequential()
model_cnn.add(Conv1D(4, kernel_size=?,input_shape=(1,lag), activation='relu',padding='valid')) 
model_cnn.add(Flatten())
model_cnn.add(Dense(1,activation='linear'))
model_cnn.compile(loss='mean_squared_error', optimizer='adam')
model_cnn.summary()
cnn_weights = model_cnn.get_weights()
model_cnn.fit(trainX, trainY, epochs=25, batch_size=1, verbose=1)

**d)** Mida desempeño de los dos modelos a través del *root mean square error* (RMSE) y compare. Para ésto deberá realizar las predicciones sobre el conjunto de entrenamiento y pruebas/validación, recuerde denormalizar los datos para que el error pueda ser computado en la escala original del rango de valores.

In [ ]:
trainPredict = model.predict(trainX,batch_size=batch_size)
trainPredict = scaler.inverse_transform(trainP)
trainY = scaler.inverse_transform([trainY])
from sklearn.metrics import mean_squared_error
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

**e)** Grafique las predicciones del conjunto de entrenamiento y pruebas/validación, y contrástelas con la serie de tiempo original. Muestre un extracto de la predicción para ver en mas detalle cómo es la predicción del modelo, comente.

In [ ]:
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataframe.values)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[lag:len(trainPredict)+lag, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataframe.values)
testPredictPlot[:, :] = np.nan
testPredictPlot[(len(trainPredict)+2*lag):, :] = testPredic

**f)** En lugar de aumentar el número de dimensiones como el el paso c), entrene la red con un *timestep=lag* y con dimensión de entrada = 1. Compare y comente sobre el cambio de dimensiones y número de parámetros de las redes ¿Cuál tipo de red tiene más parámetros y porqué? ¿Se produce una mejora del error de entrenamiento y pruebas? ¿Los tiempos de computación son comparables? Comente brevemente sobre cuál es la forma correcta para aprovechar la información a través del tiempo, si con esta forma o la realizada en el paso e).

In [ ]:
trainX = numpy.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
model_rnn = Sequential()
model_rnn.add(SimpleRNN(units=4, input_shape=(lag,1), activation='tanh',return_sequences=False,go_backwards=False))
model_rnn.add(Dense(1,activation='linear'))
model_rnn.compile(loss='mean_squared_error', optimizer='adam')
model_rnn.fit(trainX, trainY, epochs=25, batch_size=1, verbose=1)
model_rnn.summary()

#and now CNN
model_cnn = Sequential()
model_cnn.add(Conv1D(4, kernel_size=3,input_shape=(lag,1), activation='relu',padding='valid')) # or kernel=2
model_cnn.add(Flatten())
model_cnn.add(Dense(1,activation='linear'))
model_cnn.summary()

**g)** Compare el desempeño de ambos modelos al variar el *lag* definido en c), recuerde que puede variarlo entre 1 y $T_{max}$. Comente brevemente sobre qué resulta mejor, el tener más información para predecir o si ésto satura la predicción por el efecto del gradiente desvaneciente ¿Ésto ocurre sólo con el modelo recurrente o con el convolucional también? ¿Por qué?

**h)** Como habrá notado, si usa una ventana de tiempo/*timesteps* muy larga ($T$ grande), las redes tienen dificultades para aprender. Para abordar este problema se le pide experimentar con la siguiente idea: dividir una secuencia de largo $T$ en trozos de largo $K$ y representarla como una matriz de $T/K$ columnas (asumamos que $T$ es un múltiplo de $K$) y $K$ filas, y entrenar la red para procesar la secuencia formada las columnas de esa matriz. De este modo, la red debe aprender dependencias temporales más cortas (largo $T/K$ en vez de $T$), pero accede a toda la información original en forma de atributos ($K$). Llamaremos a esta idea “**time folding**”. Se le recomienda usar un valor de $K$ que tenga sentido en el problema (por ejemplo $K=7$ representaría una semana). Defina y entrene los modelos para ésta nueva representación.  
¿Cómo se ven afectados los modelos?

In [ ]:
# model_rnn = ?
# model_cnn = ?

**i)** [Opcional] Experimente con variar el número de unidades de la red recurrente (RNN) y el número de filtros de las capas convolucionales. Comente

**j)** [Opcional] Experimente con variar el *width* de los filtros convolucionales ¿Cuál sería el efecto de ésto? ¿Qué limite tiene (mayor y menor)?

#### Agregar otras mediciones como input:  $CO, PM10, NO, NO_x,...,UVB, O_3$



<img src="https://image.slidesharecdn.com/scaled-ml-2018-180529212354/95/role-of-tensors-in-machine-learning-2-638.jpg?cb=1527629345" title="tensors" width="65%"/>

*Decida si continuar con la convolucional 1D o cambiar a convolucional 2D, comente sobre ésta decisión.*

**k)** Agregue las otras mediciones como nuevos valores de entrada al modelo definido en f), por lo que ahora se transforma la serie de tiempo a N-Dimensional, se tiene un *timestep=lag* con un *features*= mediciones nuevas + Ozono. Defina los nuevos modelos con el mejor *lag* encontrado en g).

In [ ]:
#ejemplo:
new_sequences = np.vstack([co_seq,pm10_seq,...,O3_seq]) # (Contaminantes, dias de verano entre 2003-2016)

**l)** Entrene los modelos  ¿Se observan los mismos resultados? ¿Las mediciones incorporadas como información adicional ayudan al modelo? Comente y compare.

<a id="refs"></a>
## Referencias
[1] Borucki, W. J., Koch, D. G., Basri, G., Batalha, N., Boss, A., Brown, T. M., ... & Dunham, E. W. (2011). *Characteristics of Kepler planetary candidates based on the first data set*. The Astrophysical Journal, 728(2), 117. also in: https://exoplanetarchive.ipac.caltech.edu/index.html  
[2] Bugueno, M., Mena, F., & Araya, M. *Refining Exoplanet Detection Using Supervised Learning and Feature Engineering*.  
[3] Sutskever, I., Martens, J., Dahl, G. E., & Hinton, G. E. (2013). *On the importance of initialization and momentum in deep learning*. ICML (3), 28(1139-1147), 5.  
[4] Lin, T. Y., Goyal, P., Girshick, R., He, K., & Dollár, P. (2017). *Focal loss for dense object detection*. In Proceedings of the IEEE international conference on computer vision (pp. 2980-2988).  
[5] Chen, P., Chen, Y., & Rao, M. (2008). *Metrics defined by Bregman divergences: Part 2*. Communications in Mathematical Sciences, 6(4), 927-948.  
[6] Glorot, X., & Bengio, Y. (2010, March). *Understanding the difficulty of training deep feedforward neural networks*. In Proceedings of the thirteenth international conference on artificial intelligence and statistics (pp. 249-256).   
[7] He, K., Zhang, X., Ren, S., & Sun, J. (2015). *Delving deep into rectifiers: Surpassing human-level performance on imagenet classification*. In Proceedings of the IEEE international conference on computer vision (pp. 1026-1034).    
[8] Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I., & Salakhutdinov, R. (2014). *Dropout: a simple way to prevent neural networks from overfitting*. The Journal of Machine Learning Research, 15(1), 1929-1958.  
[9] Ioffe, S., & Szegedy, C. (2015). Batch normalization: *Accelerating deep network training by reducing internal covariate shift*. arXiv preprint arXiv:1502.03167.  
[10] Krizhevsky, A., Nair, V., & Hinton, G. (2014). *The CIFAR-10 dataset*. online: http://www.cs.toronto.edu/kriz/cifar.html , 4.  
[11] Bengio, Y. (2012, June). *Deep learning of representations for unsupervised and transfer learning*. In Proceedings of ICML Workshop on Unsupervised and Transfer Learning (pp. 17-36).  
[12] Simonyan, K., & Zisserman, A. (2014). *Very deep convolutional networks for large-scale image recognition*. arXiv preprint arXiv:1409.1556.  
[13] Ruder, S. (2017). *An overview of multi-task learning in deep neural networks*. arXiv preprint arXiv:1706.05098.  